In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("belsonraja/face-mask-dataset-with-and-without-mask")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\Olaoye Boluwatife\.cache\kagglehub\datasets\belsonraja\face-mask-dataset-with-and-without-mask\versions\3


In [4]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [5]:
# Path to dataset
dataset_path = r"C:\Users\Olaoye Boluwatife\.cache\kagglehub\datasets\belsonraja\face-mask-dataset-with-and-without-mask\versions\3"

In [6]:
# Create directories for train, val, test splits
base_dir = r"C:\Users\Olaoye Boluwatife\.cache\kagglehub\datasets\belsonraja\face-mask-dataset-with-and-without-mask\versions\3"  # Replace with your desired output path
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")
test_dir = os.path.join(base_dir, "test")

In [7]:
# Ensure the directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [8]:
# Class subdirectories (with_mask, without_mask)
classes = ["with_mask", "without_mask"]
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

# Split data into train, val, and test sets
for cls in classes:
    cls_path = os.path.join(dataset_path, cls)
    images = os.listdir(cls_path)
    images = [os.path.join(cls_path, img) for img in images]

    # Split into train, val, test (70%, 15%, 15%)
    train_images, temp_images = train_test_split(images, test_size=0.3, random_state=42)
    val_images, test_images = train_test_split(temp_images, test_size=0.5, random_state=42)

    # Move files to respective directories
    for img in train_images:
        shutil.copy(img, os.path.join(train_dir, cls))
    for img in val_images:
        shutil.copy(img, os.path.join(val_dir, cls))
    for img in test_images:
        shutil.copy(img, os.path.join(test_dir, cls))

print("Dataset split into train, val, and test sets.")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\Olaoye Boluwatife\\.cache\\kagglehub\\datasets\\belsonraja\\face-mask-dataset-with-and-without-mask\\versions\\3\\with_mask'

In [9]:
# Data Augmentation for Training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

In [ ]:
# Validation and Test Data Generators
val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
# Data Generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary"
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary"
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary"
)

In [ ]:
# Model Definition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification
])

In [ ]:
# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
model.summary()

In [ ]:
# Evaluate the Model
loss, accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {accuracy:.2f}")

In [ ]:
# Save the Model
model.save('mask_detector_binary.h5')


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Directory containing new images
new_images_dir = 'C:\\Users\\Olaoye Boluwatife\\.cache\\kagglehub\\datasets\\belsonraja\\face-mask-dataset-with-and-without-mask\\versions\\3\\test\\with_mask\\with_mask (8).jpg'

# Create a data generator for the new images
data_gen = ImageDataGenerator(rescale=1.0 / 255)

new_images_generator = data_gen.flow_from_directory(
    new_images_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode=None,  # No labels since we are predicting
    shuffle=False
)

# Predict
predictions = model.predict(new_images_generator)

# Interpret results
for i, pred in enumerate(predictions):
    if pred[0] > 0.5:
        print(f"Image {i + 1}: Without Mask")
    else:
        print(f"Image {i + 1}: With Mask")
